<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/GogoResNetCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnits'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (ResNet50 without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    # Split the data
    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    # Use the training data for both training and validation generators
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',  # Use train indices for validation too
        shuffle=True
    )

    # Build the model
    model = build_model()

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,  # Adjust the number of epochs
        validation_data=validation_generator,
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/10
4/4 [==============================] - 87s 22s/step - loss: 1.3647 - accuracy: 0.4643 - val_loss: 0.9994 - val_accuracy: 0.5446
Epoch 2/10
4/4 [==============================] - 71s 20s/step - loss: 0.9774 - accuracy: 0.5446 - val_loss: 0.8646 - val_accuracy: 0.4554
Epoch 3/10
4/4 [==============================] - 62s 17s/step - loss: 0.8916 - accuracy: 0.4643 - val_loss: 0.7022 - val_accuracy: 0.5446
Epoch 4/10
4/4 [==============================] - 69s 22s/step - loss: 0.7671 - accuracy: 0.5446 - val_loss: 0.6855 - val_accuracy: 0.5893
Epoch 5/10
4/4 [==============================] - 71s 20s/step - loss: 0.7332 - accuracy: 0.4643 - val_loss: 0.6852 - val_accuracy: 0.5446
Epoch 6/10
4/4 [==============================] - 70s 22s/step - loss: 0.7610 - accuracy: 0.5536 - val_loss:

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASdata'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (ResNet50 without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())  # Add Batch Normalization
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Add Dropout for regularization
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    model = build_model()

    # Fine-tuning: Unfreeze some layers
    for layer in model.layers[0].layers[-20:]:
        layer.trainable = True

    # Learning rate scheduling
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,  # Increase epochs
        validation_data=validation_generator,
        callbacks=[lr_schedule],  # Add learning rate scheduling
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/20
4/4 [==============================] - 77s 22s/step - loss: 0.8387 - accuracy: 0.5179 - val_loss: 0.7022 - val_accuracy: 0.5536 - lr: 1.0000e-04
Epoch 2/20
4/4 [==============================] - 71s 21s/step - loss: 0.7572 - accuracy: 0.5714 - val_loss: 0.6836 - val_accuracy: 0.5089 - lr: 1.2589e-04
Epoch 3/20
4/4 [==============================] - 52s 15s/step - loss: 0.7908 - accuracy: 0.6071 - val_loss: 1.9421 - val_accuracy: 0.4643 - lr: 1.5849e-04
Epoch 4/20
4/4 [==============================] - 72s 20s/step - loss: 0.8913 - accuracy: 0.5357 - val_loss: 2.6288 - val_accuracy: 0.4554 - lr: 1.9953e-04
Epoch 5/20
4/4 [==============================] - 71s 20s/step - loss: 0.9743 - accuracy: 0.5089 - val_loss: 2.7733 - val_accuracy: 0.4554 - lr: 2.5119e-04
Epoch 6/20
4/4 [==============================] - 71s 20s/step - loss: 0.8478 - accuracy

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnited'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (ResNet50 without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())  # Add Batch Normalization
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Add Dropout for regularization
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    model = build_model()

    # Fine-tuning: Unfreeze some layers
    for layer in model.layers[0].layers[-20:]:
        layer.trainable = True

    # Learning rate scheduling
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,  # Increase epochs
        validation_data=validation_generator,
        callbacks=[lr_schedule],  # Add learning rate scheduling
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/20
4/4 [==============================] - 94s 22s/step - loss: 0.5920 - accuracy: 0.7321 - val_loss: 0.6738 - val_accuracy: 0.8214 - lr: 1.0000e-04
Epoch 2/20
4/4 [==============================] - 71s 21s/step - loss: 0.3542 - accuracy: 0.8661 - val_loss: 0.7296 - val_accuracy: 0.8214 - lr: 1.2589e-04
Epoch 3/20
4/4 [==============================] - 68s 20s/step - loss: 0.3843 - accuracy: 0.9018 - val_loss: 0.6894 - val_accuracy: 0.8214 - lr: 1.5849e-04
Epoch 4/20
4/4 [==============================] - 69s 21s/step - loss: 0.6360 - accuracy: 0.8482 - val_loss: 0.4061 - val_accuracy: 0.8571 - lr: 1.9953e-04
Epoch 5/20
4/4 [==============================] - 59s 17s/step - loss: 0.4656 - accuracy: 0.8304 - val_loss: 0.4620 - val_accuracy: 0.8393 - lr: 2.5119e-04
Epoch 6/20
4/4 [=======

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold

# Define paths to your dataset
data_dir = '/content/drive/MyDrive/MIASUnited'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load and augment data
datagen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Extract labels for StratifiedKFold
labels = datagen.labels

# Convert labels to integers (if needed)
labels = np.array(labels, dtype=int)

# Define the base model (ResNet50 without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Custom model for breast cancer detection
def build_model():
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())  # Add Batch Normalization
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Add Dropout for regularization
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# K-Fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
final_accuracy = []

for train_index, val_index in skf.split(datagen.filenames, labels):
    print(f"Fold {fold}/5")

    train_data = np.array(datagen.filenames)[train_index]
    val_data = np.array(datagen.filenames)[val_index]

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )

    model = build_model()

    # Fine-tuning: Unfreeze some layers
    for layer in model.layers[0].layers[-20:]:
        layer.trainable = True

    # Learning rate scheduling
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,  # Increase epochs
        validation_data=validation_generator,
        callbacks=[lr_schedule],  # Add learning rate scheduling
        verbose=1
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    final_accuracy.append(test_acc)

    fold += 1

# Print final results
print("Final Accuracy:", np.mean(final_accuracy))


Found 112 images belonging to 2 classes.
Fold 1/5
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/20
4/4 [==============================] - 81s 22s/step - loss: 0.6633 - accuracy: 0.7679 - val_loss: 1.0481 - val_accuracy: 0.8214 - lr: 1.0000e-04
Epoch 2/20
4/4 [==============================] - 71s 21s/step - loss: 0.4942 - accuracy: 0.8214 - val_loss: 1.1701 - val_accuracy: 0.8214 - lr: 1.2589e-04
Epoch 3/20
4/4 [==============================] - 71s 22s/step - loss: 0.3922 - accuracy: 0.8661 - val_loss: 0.8484 - val_accuracy: 0.8214 - lr: 1.5849e-04
Epoch 4/20
4/4 [==============================] - 73s 20s/step - loss: 0.4768 - accuracy: 0.8304 - val_loss: 0.8099 - val_accuracy: 0.8214 - lr: 1.9953e-04
Epoch 5/20
4/4 [==============================] - 58s 16s/step - loss: 0.3703 - accuracy: 0.8750 - val_loss: 0.8362 - val_accuracy: 0.8304 - lr: 2.5119e-04
Epoch 6/20
4/4 [==============================] - 73s 22s/step - loss: 0.5193 - accuracy